Wine review
===========

This notebook contains all the code for the wine review problem.
As the dataset have a "points" field that gives notation of the wine, we will try to perform a regression task to predict the score of a wine.

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# load our train and test datasets
initial_df = pd.read_csv("datasets/wine_reviews/winemag-data-130k-v2.csv")